In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager
import matplotlib.pyplot as plt
import requests
from splinter import Browser
import time

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/sabrinaalves/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


In [3]:
url =  "https://music.apple.com/us/playlist/top-100-usa/pl.606afcbb70264d2eb2b51d8dbcfa6a12"
browser.visit(url)
time.sleep(5)
html = browser.html

In [4]:
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [5]:
#make the indentation proper
print(soup.prettify())

<html class="hydrated" dir="ltr" hydrated="" lang="en-US" prefix="og: http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <style data-styles="">
   apple-podcast-player,apple-music-uploaded-content,apple-tv-plus-player,apple-tv-plus-preview,apple-music-video-player,amp-chrome-player,amp-footer-player,apple-music-card-player,amp-lyrics,amp-episode-list-control,amp-next-episode-control,apple-music-artwork-lockup,apple-music-progress,apple-music-volume,amp-playback-controls-autoplay,apple-music-radio-column-player,amp-ambient-video,amp-background-video,amp-footer-player-metadata,amp-footer-player-progress,amp-launch-button,amp-mediakit-root,amp-playback-controls-pip,amp-podcast-keyboard-controls,amp-text-multiline,amp-transition,state-holder,amp-chrome-volume,apple-music-artwork,amp-popover,amp-popover-item,amp-embed-empty-state,amp-volume-control,amp-lyrics-display-instrumental-line,amp-lyrics-display-synced-line,amp-lyrics-display-footer,amp-playback-controls-progress-range,amp-icon

In [6]:
#Fetch all the table tags
all_table = soup.find_all('table')
print(all_table)

[]


In [7]:
songs = soup.find_all('div', {"class": 'songs-list-row'})
song = songs[0]
song

<div class="songs-list-row songs-list-row--web-preview web-preview songs-list-row--two-lines songs-list-row--song" data-metrics-click="" data-row="0" role="row" tabindex="0">
<div class="songs-list__col songs-list__col--heart-and-star typography-body">
<div class="love-or-popular">
<button aria-check="false" aria-label="Loved" class="love-or-popular__glyph" disabled="" role="checkbox" title="">
</button>
</div>
</div>
<div class="songs-list__col songs-list__col--song typography-body">
<div class="songs-list-row__song-container">
<!-- --> <div class="songs-list-row__song-wrapper">
<div class="songs-list-row__song-name-wrapper songs-list-row__song-name-wrapper--explicit">
<div class="songs-list-row__explicit-wrapper" dir="auto">
<div aria-checked="false" class="songs-list-row__song-name" dir="auto" role="checkbox" tabindex="-1">What Happened To Virgil (feat. Gunna)</div>
<span aria-label="Explicit" class="badge explicit default songs-list-row__badge" role="img">
<svg aria-hidden="true" c

In [8]:
len(songs)

100

In [9]:
song_name = song.find("div",{"class": "songs-list-row__song-name"}).text.strip()
song_name

'What Happened To Virgil (feat. Gunna)'

In [10]:
artist_name = song.find("div",{"class": "songs-list__col--artist"}).text.strip()
artist_name

'Lil Durk'

In [11]:
album_name = song.find("div",{"class": "songs-list__col--album"}).text.strip()
album_name

'7220'

In [12]:
song_length = song.find("div",{"class": "songs-list__col--time"}).text.strip().split("\n")[0]
song_length

'3:01'

In [13]:
for i in range(len(songs)):
    rank = i + 1
    song = songs[i]

In [14]:
song_data = {
        "song_name": song_name,
        "artist_name": artist_name,
        "album_name": album_name,
        "song_length": song_length
    }

In [20]:
song_names=[]
artists=[]
albums=[]
song_lengths=[]
rank = 0
rank_list=[]

for song in songs:
    song_name = song.find("div",{"class": "songs-list-row__song-name"}).text.strip()
    artist_name = song.find("div",{"class": "songs-list__col--artist"}).text.strip().split("\n")[0]
    album_name = song.find("div",{"class": "songs-list__col--album"}).text.strip()
    song_length = song.find("div",{"class": "songs-list__col--time"}).text.strip().split("\n")[0]
    rank +=1
    
    
    song_names.append(song_name)
    artists.append(artist_name)
    albums.append(album_name)
    song_lengths.append(song_length)
    rank_list.append(rank)
    

In [25]:
apple= pd.DataFrame()

apple["apple_rank"]= rank_list
apple["song_name"]= song_names
apple["artist"]= artists
apple["albums"]= albums
apple["song_length"]= song_lengths
apple["apple_date_retrieved"] = "03/20/2022"
apple["provider"] = "apple"
apple.head()

,apple_rank,song_name,artist,albums,song_length,apple_date_retrieved,provider
0,1,What Happened To Virgil (feat. Gunna),Lil Durk,7220,3:01,03/20/2022,apple
1,2,AHHH HA,Lil Durk,7220,3:06,03/20/2022,apple
2,3,Super Gremlin,Kodak Black,Sniper Gang Presents Syko Bob & Snapkatt: Nigh...,3:20,03/20/2022,apple
3,4,No Interviews,Lil Durk,7220,2:59,03/20/2022,apple
4,5,Petty Too (feat. Future),Lil Durk,7220,2:39,03/20/2022,apple


In [26]:
#create csv
apple.to_csv("apple_data.csv")